In [ ]:
from google.colab import drive

drive.mount('/drive')

Mounted at /drive


In [ ]:
#!git clone https://github.com/lancopku/codable-watermarking-for-llm
!pip install -q transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.7 MB/s eta 0:00:00


In [ ]:
cd '/drive/My Drive/Colab Notebooks/ISO/'

/drive/My Drive/Colab Notebooks/ISO


In [ ]:
cd wang-codable-watermarking-for-llm

/drive/My Drive/Colab Notebooks/ISO/wang-codable-watermarking-for-llm


In [ ]:
!pip install -q -r new_requirements.txt
!pip install -q importlib datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from importlib import reload
import torch
import pandas as pd
from tqdm import tqdm
import os
import watermarking.watermark_processor

from watermarking.watermark_processor import RepetitionPenaltyLogitsProcessor
from transformers import LogitsProcessorList, MinLengthLogitsProcessor, LogitsProcessor, AutoTokenizer, AutoModelForCausalLM
from watermarking.utils.text_tools import truncate
from watermarking.utils.load_local import load_local_model_or_tokenizer
from datasets import load_dataset, load_from_disk

import watermarking.watermark_processors.message_models.lm_message_model
import watermarking.watermark_processors.message_model_processor

from watermarking.watermark_processors.message_models.lm_message_model import LMMessageModel
from watermarking.watermark_processors.message_model_processor import WmProcessorMessageModel

In [ ]:
checkpoint = 'facebook/opt-1.3b'

model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto")

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [ ]:
min_length_processor = MinLengthLogitsProcessor(min_length=1000,
                                                eos_token_id=tokenizer.eos_token_id)
repetition_processor = RepetitionPenaltyLogitsProcessor(penalty=1.5)

watermarking.watermark_processor = reload(watermarking.watermark_processor)

watermarking.watermark_processors.message_models.lm_message_model = reload(watermarking.watermark_processors.message_models.lm_message_model)

watermarking.watermark_processors.message_model_processor = reload(watermarking.watermark_processors.message_model_processor)

In [ ]:
cd '../instances'

/drive/MyDrive/Colab Notebooks/ISO/instances


In [ ]:
lm_message_model = LMMessageModel(tokenizer=tokenizer, lm_model=model, lm_tokenizer=tokenizer,
    delta = 1.1, lm_prefix_len=10, lm_topk=-1, message_code_len = 20, random_permutation_num=50)
wm_precessor_message_model = WmProcessorMessageModel(message_model=lm_message_model, tokenizer=tokenizer,
    encode_ratio=5, max_confidence_lbd=0.5, strategy='max_confidence', message=[42,34,54,665,226,329])

In [ ]:
def calculate_scores(filename, keyword, custom_filename=None):
    detection_filename = (filename[:-5] + '_detection.json') if custom_filename == None else custom_filename
    df = pd.read_json(filename, lines=True)
    with open(detection_filename, "r+" if os.path.exists(detection_filename) else "w+") as f:
        with torch.no_grad():
            for i in tqdm(range(len(f.readlines()), len(df["input"]))):
                line = (df["input"][i] if keyword == "continuation" else "") + df[keyword][i]

                decoded_message, other_information = wm_precessor_message_model.decode(line, disable_tqdm=True)
                confidence = other_information[1][-1][0]
                message = tokenizer.decode(decoded_message)

                _line = df["input"][i].replace('\n', '\\n').replace('"', '\\"')
                _message = message.replace('\n', '\\n').replace('"', '\\"')

                f.write(f'{{"input": "{_line}", "confidence": "{confidence}", "message": "{_message}"}}\n')

In [ ]:
calculate_scores('wang_no_attack.json', 'continuation')

100%|██████████| 512/512 [1:03:41<00:00,  7.46s/it]


In [ ]:
calculate_scores('wang_paraphrase.json', 'paraphrase')

100%|██████████| 512/512 [51:48<00:00,  6.07s/it]


In [ ]:
calculate_scores('wang_backtranslated.json', 'translation')

100%|██████████| 57/57 [07:00<00:00,  7.37s/it]


In [ ]:
calculate_scores('c4_selection.json', 'continuation', 'wang_c4_detection.json')

100%|██████████| 512/512 [55:48<00:00,  6.54s/it]


# Tests

In [ ]:
calculate_scores('wang_no_attack.json', 'continuation')

  0%|          | 1/512 [00:08<1:13:56,  8.68s/it]

90


  0%|          | 2/512 [00:16<1:09:54,  8.22s/it]

83


  1%|          | 3/512 [00:24<1:08:58,  8.13s/it]

...

86


  3%|▎         | 13/512 [01:40<1:03:04,  7.58s/it]

52


  3%|▎         | 13/512 [01:42<1:05:25,  7.87s/it]


KeyboardInterrupt: 

In [ ]:
calculate_scores('wang_paraphrase.json', 'paraphrase')

  0%|          | 1/512 [00:06<59:06,  6.94s/it]

54


  0%|          | 2/512 [00:11<48:57,  5.76s/it]

67


  1%|          | 3/512 [00:19<55:45,  6.57s/it]

...

57


  4%|▍         | 20/512 [02:03<52:53,  6.45s/it]

31


  4%|▍         | 20/512 [02:06<52:02,  6.35s/it]


KeyboardInterrupt: 

In [ ]:
calculate_scores('c4_selection.json', 'continuation', 'wang_c4_detection.json')

  0%|          | 1/512 [00:06<54:30,  6.40s/it]

37


  0%|          | 2/512 [00:12<55:19,  6.51s/it]

39


  1%|          | 3/512 [00:19<54:59,  6.48s/it]

...

36


  3%|▎         | 15/512 [01:37<54:27,  6.57s/it]

38


  3%|▎         | 15/512 [01:41<56:16,  6.79s/it]


KeyboardInterrupt: 

In [ ]:
calculate_scores('wang_no_attack.json', False)

  0%|          | 0/512 [00:07<?, ?it/s]

[199505, 73531, 4]
[(85, 0, 0.05466558039188385), (91, 2, 0.17155452072620392), (90, 0, 5.182834684092086e-06)]

Detected: 0
Not detected: 0


In [ ]:
decoded_message, other_information = wm_precessor_message_model.decode("hello, this is a sample text to check what the method is actually outputing", disable_tqdm=True)

In [ ]:
decoded_message

[79]

In [ ]:
other_information[1]

[(6, 0, 7.2940151767397765e-06)]

In [ ]:
p_value = (100. - other_information[1][0][0]) / 100.